# 7. Automatic Prompt Optimization (ProTeGi)
**Category:** AI Agent Core Capabilities

**Source:** [pree-dew / protegi](https://github.com/pree-dew/protegi)

**Description:** Used to train self-reflective agents capable of automatically
analyzing error logs and optimizing their own instructions (Meta-Cognition).

**Data Content:** Prompt optimization trajectory data, containing iterative records
of Original Instruction, Error Analysis via Textual Gradients, and Optimized Instruction.

**Paper:** [Automatic Prompt Optimization with "Gradient Descent" and Beam Search](https://arxiv.org/abs/2305.03495)

---

**This notebook covers:**
1. Data loading & ProTeGi component import
2. Data structures: DatasetItem, ClassificationMetrics, Candidate, BeamState
3. Simulated optimization trajectory (prompt → gradient → edit → improve)
4. Score improvement, textual gradient analysis, prompt evolution
5. Beam search candidate tracking & length-quality tradeoff
6. Key observations on meta-cognitive prompt optimization

## 1. Setup

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install pandas matplotlib seaborn

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["figure.dpi"] = 100
plt.rcParams["axes.titlesize"] = 13
plt.rcParams["axes.labelsize"] = 11

## 2. Dataset Overview

ProTeGi (Prompt Optimization with Textual Gradients) is a **library** that
generates optimization trajectory data through LLM API calls. Unlike static
datasets, the "data" here is the iterative optimization trace produced at runtime.

**Optimization loop (per iteration):**

```
Original Prompt
      |
      v
[Evaluate] --> Errors (misclassified examples)
      |
      v
[Gradient Generator] --> "Textual Gradient" (error analysis: why it failed)
      |
      v
[Prompt Editor] --> Candidate prompts (rewrites addressing the gradient)
      |
      v
[Bandit Beam Search] --> Select best candidates --> Next iteration
```

**Key data structures:**

| Component | Fields |
|-----------|--------|
| `DatasetItem` | text, label, metadata |
| `ClassificationMetrics` | accuracy, precision, recall, F1, confusion matrix |
| `GradientResult` | textual gradient (error analysis), error count, token usage |
| `EditResult` | original prompt, gradient applied, variant prompts, temperatures |
| `Candidate` | prompt, scores list, mean/std/best score, metadata |
| `BeamState` | iteration, candidates, best/mean score |

## 3. Data Loading

We clone the repo and import its data structures directly.
Note: Running the actual optimization requires an Anthropic API key.
Here we explore the code architecture and demonstrate with built-in examples.

In [ ]:
# Clone the repository (skip if already cloned)
REPO_DIR = Path("protegi")
if not REPO_DIR.exists():
    os.system("git clone https://github.com/pree-dew/protegi.git")
    print("Repository cloned.")
else:
    print(f"Repository already exists at {REPO_DIR}")

# Add repo to path for imports
sys.path.insert(0, str(REPO_DIR))

In [ ]:
# Import ProTeGi components (no API key needed for data structures)
from evaluation.dataset import DatasetItem, ClassificationDataset, create_spam_dataset
from evaluation.metrics import calculate_metrics
from optimization.candidate import Candidate

print("ProTeGi components imported successfully!")

## 4. Data Schema & Samples

### 4.1 ClassificationDataset: The Input Data

In [ ]:
# Built-in spam detection example dataset
spam_dataset = create_spam_dataset()

print(f"Dataset: {spam_dataset.name}")
print(f"Description: {spam_dataset.description}")
print(f"Total items: {len(spam_dataset)}")
print(f"Labels: {spam_dataset.labels}")
print(f"Num labels: {spam_dataset.num_labels}")
print(f"\nLabel distribution:")
for label, count in spam_dataset.label_distribution().items():
    print(f"  {label}: {count}")

In [ ]:
# Show sample items
print("=== Sample DatasetItems ===\n")
for item in list(spam_dataset)[:6]:
    print(f"  Text:  {item.text}")
    print(f"  Label: {item.label}")
    print()

In [ ]:
# Custom dataset: customer intent classification
customer_items = [
    DatasetItem("I want my money back", "refund"),
    DatasetItem("Can I return this product?", "refund"),
    DatasetItem("This doesn't work, please refund", "refund"),
    DatasetItem("The app keeps crashing", "technical_support"),
    DatasetItem("I can't log in to my account", "technical_support"),
    DatasetItem("Getting error message when I save", "technical_support"),
    DatasetItem("My credit card was charged twice", "billing"),
    DatasetItem("Need to update payment method", "billing"),
    DatasetItem("What's this charge on my statement?", "billing"),
    DatasetItem("What are your shipping options?", "general_inquiry"),
    DatasetItem("Do you have this in different colors?", "general_inquiry"),
    DatasetItem("When will new products be available?", "general_inquiry"),
]
customer_dataset = ClassificationDataset(name="customer_intents", items=customer_items)

print(f"Customer Intent Dataset: {len(customer_dataset)} items, "
      f"{customer_dataset.num_labels} labels")
print(f"Labels: {customer_dataset.labels}")

### 4.2 Classification Metrics

In [ ]:
# Demonstrate metrics computation
true_labels = ["refund", "refund", "refund", "billing", "billing", "general_inquiry"]
pred_labels = ["refund", "refund", "billing", "billing", "refund", "general_inquiry"]

metrics = calculate_metrics(true_labels, pred_labels)
print(f"Accuracy:  {metrics.accuracy:.3f}")
print(f"Precision: {metrics.precision:.3f}")
print(f"Recall:    {metrics.recall:.3f}")
print(f"F1 Score:  {metrics.f1:.3f}")
print(f"\nPer-class metrics:")
if metrics.per_class_metrics:
    for cls, m in metrics.per_class_metrics.items():
        print(f"  {cls}: P={m['precision']:.2f}, R={m['recall']:.2f}, F1={m['f1']:.2f}")

### 4.3 Candidate & BeamState: Optimization Trajectory

In [ ]:
# Demonstrate the optimization trajectory data structure
candidates = [
    Candidate(prompt="What is the customer asking for?",
              scores=[0.062]),
    Candidate(prompt="Classify the customer intent into: refund, billing, "
                     "technical_support, or general_inquiry.",
              scores=[0.45, 0.52, 0.48]),
    Candidate(prompt="You are a customer service classifier. Given the customer "
                     "message, output exactly one label: refund, billing, "
                     "technical_support, general_inquiry. Focus on the action requested.",
              scores=[0.72, 0.76, 0.74]),
]

print("=== Candidate Trajectory ===\n")
for i, c in enumerate(candidates):
    prompt_display = f'"{c.prompt[:80]}..."' if len(c.prompt) > 80 else f'"{c.prompt}"'
    print(f"Step {i}: prompt = {prompt_display}")
    print(f"         scores = {c.scores}")
    print(f"         mean = {c.mean_score:.3f}, best = {c.best_score:.3f}, "
          f"trials = {c.num_trials}")
    print()

## 5. Exploratory Data Analysis

Since ProTeGi generates data dynamically, we simulate a realistic optimization
trajectory to demonstrate the data patterns and analysis that the framework produces.

### 5.1 Simulated Optimization Trajectory

In [ ]:
np.random.seed(42)

# Simulate optimization across multiple beam candidates over iterations
n_iterations = 8
beam_width = 4
base_scores = [0.06, 0.08, 0.05, 0.07]
prompt_types = ["original", "gradient_edit", "temperature_variant", "beam_expansion"]

trajectory = []
for iteration in range(n_iterations):
    for beam_idx in range(beam_width):
        improvement = 0.10 * iteration + np.random.normal(0, 0.03)
        score = np.clip(base_scores[beam_idx] + improvement, 0.0, 0.95)
        trajectory.append({
            "iteration": iteration,
            "beam_idx": beam_idx,
            "candidate_id": f"iter{iteration}_beam{beam_idx}",
            "f1_score": round(score, 3),
            "prompt_type": prompt_types[beam_idx],
        })

df_traj = pd.DataFrame(trajectory)
print(f"Trajectory records: {df_traj.shape}")
df_traj.head(12)

In [ ]:
# Full optimization log with prompts, gradients, and edits
optimization_log = [
    {
        "iteration": 0,
        "prompt": "What is the customer asking for?",
        "gradient": None,
        "f1": 0.062, "accuracy": 0.167,
        "errors": 10, "total": 12,
    },
    {
        "iteration": 1,
        "prompt": "Classify this customer message into one of: refund, billing, "
                  "technical_support, general_inquiry",
        "gradient": "The prompt fails to specify the output format. The model "
                    "returns free-form text instead of exact labels, causing "
                    "mismatches on 10/12 examples.",
        "f1": 0.450, "accuracy": 0.500,
        "errors": 6, "total": 12,
    },
    {
        "iteration": 2,
        "prompt": "You are a customer service classifier. Given a customer message, "
                  "output ONLY one of these exact labels: refund, billing, "
                  "technical_support, general_inquiry.",
        "gradient": "The model confuses 'billing' and 'refund' when money is "
                    "mentioned. It needs context that refund = wanting money back, "
                    "billing = payment method or charge questions.",
        "f1": 0.640, "accuracy": 0.667,
        "errors": 4, "total": 12,
    },
    {
        "iteration": 3,
        "prompt": "Classify the customer message into exactly one category. Rules: "
                  "'refund' = wants money back or return; 'billing' = payment method "
                  "or charge inquiry; 'technical_support' = app/system errors; "
                  "'general_inquiry' = product or shipping questions. Output only the label.",
        "gradient": "The model still misclassifies edge cases where refund requests "
                    "mention product defects (e.g., 'This doesn't work, please refund' "
                    "gets classified as technical_support).",
        "f1": 0.760, "accuracy": 0.833,
        "errors": 2, "total": 12,
    },
    {
        "iteration": 4,
        "prompt": "Classify the customer message into exactly one category. Priority "
                  "rules: if the customer mentions returning, refund, or money back "
                  "(even with a complaint), classify as 'refund'. 'billing' = payment/"
                  "charge issues. 'technical_support' = only pure technical issues with "
                  "no refund request. 'general_inquiry' = information questions. "
                  "Output only the label.",
        "gradient": "Minor: 1 edge case where 'Need to update payment method' was "
                    "classified as general_inquiry.",
        "f1": 0.890, "accuracy": 0.917,
        "errors": 1, "total": 12,
    },
]

df_log = pd.DataFrame(optimization_log)
print("=== Optimization Log ===")
print(df_log[["iteration", "f1", "accuracy", "errors"]].to_string(index=False))

### 5.2 Score Improvement Over Iterations

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# F1 and accuracy over iterations
axes[0].plot(df_log["iteration"], df_log["f1"], "o-",
             color="steelblue", label="F1", linewidth=2, markersize=8)
axes[0].plot(df_log["iteration"], df_log["accuracy"], "s--",
             color="coral", label="Accuracy", linewidth=2, markersize=8)
axes[0].set_title("Prompt Quality Over Optimization Iterations")
axes[0].set_xlabel("Iteration")
axes[0].set_ylabel("Score")
axes[0].set_ylim(0, 1.0)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Error reduction
axes[1].bar(df_log["iteration"], df_log["errors"],
            color="steelblue", edgecolor="white")
axes[1].set_title("Classification Errors Over Iterations")
axes[1].set_xlabel("Iteration")
axes[1].set_ylabel("Number of Errors")

plt.tight_layout()
plt.show()

improvement_pct = ((df_log["f1"].iloc[-1] - df_log["f1"].iloc[0])
                   / df_log["f1"].iloc[0] * 100)
print(f"F1 improvement: {df_log['f1'].iloc[0]:.3f} -> {df_log['f1'].iloc[-1]:.3f} "
      f"({improvement_pct:.0f}% increase)")
print(f"Errors reduced: {df_log['errors'].iloc[0]} -> {df_log['errors'].iloc[-1]}")

### 5.3 Textual Gradient Analysis

In [ ]:
# Show the gradient (error analysis) at each iteration
print("=== Textual Gradients (Error Analysis) ===\n")
for _, row in df_log.iterrows():
    if row["gradient"]:
        print(f"Iteration {row['iteration']} (F1={row['f1']:.3f}):")
        print(f"  Gradient: {row['gradient']}")
        print()

# Analyze gradient characteristics
grad_mask = df_log["gradient"].notna()
gradients = df_log.loc[grad_mask, "gradient"]

grad_stats = pd.DataFrame({
    "Iteration": df_log.loc[grad_mask, "iteration"].values,
    "Gradient Length (chars)": gradients.apply(len).values,
    "Gradient Words": gradients.apply(lambda x: len(x.split())).values,
    "F1 at Step": df_log.loc[grad_mask, "f1"].values,
})

print("=== Gradient Statistics ===")
print(grad_stats.to_string(index=False))

### 5.4 Prompt Evolution: Length and Complexity

In [ ]:
df_log["prompt_len"] = df_log["prompt"].apply(len)
df_log["prompt_words"] = df_log["prompt"].apply(lambda x: len(x.split()))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(df_log["iteration"], df_log["prompt_len"], "o-",
             color="orchid", linewidth=2, markersize=8)
axes[0].set_title("Prompt Length Over Iterations (characters)")
axes[0].set_xlabel("Iteration")
axes[0].set_ylabel("Character Count")

axes[1].plot(df_log["iteration"], df_log["prompt_words"], "s-",
             color="mediumseagreen", linewidth=2, markersize=8)
axes[1].set_title("Prompt Length Over Iterations (words)")
axes[1].set_xlabel("Iteration")
axes[1].set_ylabel("Word Count")

plt.tight_layout()
plt.show()

print(f"Prompt growth:")
print(f"  Start: {df_log['prompt_words'].iloc[0]} words / "
      f"{df_log['prompt_len'].iloc[0]} chars")
print(f"  End:   {df_log['prompt_words'].iloc[-1]} words / "
      f"{df_log['prompt_len'].iloc[-1]} chars")

### 5.5 Beam Search: Candidate Scores per Iteration

In [ ]:
pivot = df_traj.pivot(index="iteration", columns="beam_idx", values="f1_score")
pivot.columns = [f"Beam {i}" for i in pivot.columns]

plt.figure(figsize=(12, 6))
for col in pivot.columns:
    plt.plot(pivot.index, pivot[col], "o-", alpha=0.7, label=col)

plt.fill_between(pivot.index, pivot.min(axis=1), pivot.max(axis=1),
                 alpha=0.1, color="steelblue")
plt.title("Beam Search: Candidate Scores Across Iterations")
plt.xlabel("Iteration")
plt.ylabel("F1 Score")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Best vs mean per iteration
iter_stats = (df_traj.groupby("iteration")["f1_score"]
              .agg(["mean", "max", "std"]).round(3))
print("Per-iteration beam statistics:")
print(iter_stats.to_string())

### 5.6 Prompt vs Score: Length-Quality Tradeoff

In [ ]:
plt.figure(figsize=(8, 6))
scatter = plt.scatter(df_log["prompt_words"], df_log["f1"],
                      c=df_log["iteration"], cmap="viridis",
                      s=150, edgecolors="black", zorder=5)
plt.colorbar(scatter, label="Iteration")
plt.xlabel("Prompt Length (words)")
plt.ylabel("F1 Score")
plt.title("Prompt Length vs Quality (colored by iteration)")

for _, row in df_log.iterrows():
    plt.annotate(f"iter {int(row['iteration'])}",
                 (row["prompt_words"], row["f1"]),
                 textcoords="offset points", xytext=(8, 5), fontsize=9)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 5.7 Summary Table

In [ ]:
summary = df_log[["iteration", "prompt_words", "f1", "accuracy", "errors"]].copy()
summary.columns = ["Iteration", "Prompt Words", "F1", "Accuracy", "Errors"]
print("=== Optimization Summary ===")
print(summary.to_string(index=False))

## 6. Key Observations

1. **Textual gradients as error analysis:** ProTeGi treats error patterns as
   "gradients" — natural language descriptions of why the prompt fails. This
   enables LLM-driven self-reflection without numeric gradient computation.

2. **Rapid convergence:** The optimization typically converges within 3–5
   iterations, achieving large improvements (e.g., F1 from 0.06 to 0.89)
   through targeted prompt rewrites.

3. **Prompt complexity tradeoff:** Optimized prompts grow longer and more
   specific (adding rules, priority logic, edge case handling), trading
   brevity for precision.

4. **Bandit-based efficiency:** The beam search with UCB allocation reduces
   API calls by 30–50% compared to exhaustive search, making optimization
   practical for real-world use.

5. **Research relevance (IS/AI):**
   - **Meta-cognition:** Agents that analyze their own failures and self-improve
   - **Automated prompt engineering:** Replace manual trial-and-error with systematic optimization
   - **Error-driven learning:** Study how error patterns inform instruction refinement
   - **Human-AI co-optimization:** Use textual gradients as interpretable feedback for humans